In [1]:
!pip install pandas

In [2]:
import pandas as pd
from google.colab import drive

# Authenticate and mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import math
import numpy as np

#thanks chat gpt
def calculate_distance(num1, num2):
    grid = {
        1: (0, 0),
        2: (1, 0),
        3: (2, 0),
        4: (0, 1),
        5: (1, 1),
        6: (2, 1),
        7: (0, 2),
        8: (1, 2),
        9: (2, 2)
    }

    x1, y1 = grid[num1]
    x2, y2 = grid[num2]
    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return round(distance, 3)

In [5]:
# Original
dist2time = {
    0.0 : 0,
    1.0 : 10,
    1.414 : 22,
    2.0 : 15,
    2.236 : 34,
    2.828 : 25}

#Average
#dist2time = {
#    0.0 : 0,
#    1.0 : 11.8,
#    1.414 : 32.8,
#    2.0 : 14.9,
#    2.236 : 15.8,
#    2.828 : 22.8}

#Worst
#dist2time = {
#    0.0 : 0,
#    1.0 : 19,
#    1.414 : 33,
#    2.0 : 23,
#    2.236 : 25,
#   2.828 : 25}

In [6]:
from itertools import permutations

def generate_ordered_subsets(s):
    subsets = []
    n = len(s)
    for r in range(1, n + 1):
        subsets.extend(permutations(s, r))
    return subsets

In [10]:
def compute_score(path, printOutputs=False):
  nums = [] # array for active sites
  ops = [] #operations

  battery_coefficient = 0.2 # battery coefficient means how fast battery is used

  index = 0 # puts operations into ops and numbers into nums
  for i in range(0, len(path)//2):
    ops.append(path[index])
    nums.append(path[index+1])
    index += 2

  #from base to first site
  distance = calculate_distance(5, int(nums[0]))  # this calculates the distance from base to the first site.
  total_distance = distance # The distance you have moved so far
  time_spent = dist2time[distance] + 20 #moving and collecting
  battery_used = battery_coefficient * distance #not carrying anything
  collected = 0 # collected is the number of samples you have
  if ops[0] == '+': # if the operation is addition then you add that number to to collected
      collected += int(nums[0])
  elif ops[0] == '*': # if you don't do addition then you multiply that number by collected
      collected *= 0

  #rest of sites
  for i in range(1, len(nums)):
    distance = calculate_distance(int(nums[i-1]), int(nums[i])) # this calculates the distance from the first site to the seceond site
    total_distance += distance # this adds the distance you moved to the distance you just moved
    time_spent += dist2time[distance] # uses the distance to find the time spent
    battery_used += battery_coefficient * distance * (1 + collected) # used the distance and coefficient to find battery spent
    if ops[i] == '+': # if the operation is addition then you add that number to to collected
      collected += int(nums[i])
    elif ops[i] == '*':# if you don't do addition then you multiply that number by collected
      collected *= int(nums[i])
    time_spent += 20

  #from last site to base
  distance = calculate_distance(int(nums[-1]), 5) # calculates the distance from the seceond site to the third
  total_distance += distance # adds the currant distance to the distance you just moved
  time_spent += dist2time[distance] + 20 # uses the distance to find the amount of time you spent
  battery_used += battery_coefficient * distance * (1 + collected) # calculates the battery used with the distance

  #calculate final score
  collection_score = collected
  collection_bonus = 0
  collection_penalty = 0
  inactive_penalty = 0
  if collected > 24:
    collection_penalty = abs(24 - collected) # if you have more than 24 samples then there is a penalty
  elif collected == 24:
    collection_bonus = 10 # if you collected 24 samples then you get a collection bonus of 10
  elif collected == 0:
    inactive_penalty = 10
# This finds the battery score using the the battery penalty
  battery_penalty = 0
  battery_score = 100 - battery_used #  100 is the battery you start with at the beginning so this does 100 - the battery that you used
  if battery_score < 0: # what this does is if you battery score is less then 0, then there is a penalty⬇
    battery_penalty = abs(battery_score) * 2

  time_score = 180 - time_spent # we have 180 seceonds to complete the game so this subracts our time spent from 180

  total_score = 0.5 * battery_score + 0.05 * time_score + collection_score - (collection_penalty + battery_penalty + inactive_penalty) + collection_bonus #formula for total score
  if printOutputs: # prints out the score
    print("battery score: ", battery_score) #  battery score
    print("time score: ", time_score) #  time score
    print("collection score: ", collection_score) # collection score
    print("collection penalty: ", collection_penalty) # collection penalty. If you go over 24 samples then there is a penalty
    print("battery penalty: ", battery_penalty) # battery penalty. The more battery you use, the less score
    print("inactive penalty: ", inactive_penalty) # inactive penalty
    print("collection bonus: ", collection_bonus) # collection bonus
    print("total score: ", total_score) # total score
  return total_score

In [11]:
compute_score("+4*4+8", True)

battery score:  89.9924
time score:  58
collection score:  24
collection penalty:  0
battery penalty:  0
inactive penalty:  0
collection bonus:  10
total score:  81.8962


81.8962

In [12]:
compute_score("+3*6+6", True)

battery score:  93.9172
time score:  58
collection score:  24
collection penalty:  0
battery penalty:  0
inactive penalty:  0
collection bonus:  10
total score:  83.8586


83.8586

In [14]:
#This function computes the best possible path based on the available sites.
#In the for loop, it goes through each of the three numbers and puts in every possible action for each number.
#Using the Generate_Ordered_Subsetsfunction it generates all of the possible orders all the three numbers can be in.
#Total viable paths makes an array that holds all of the paths that will work.
#

# takes string of sites like "2 4 7"
def predictor(path):
  # list all possible actions
  extended_paths = []
  for item in path:
    if item.isdigit():
      extended_paths.append("+" + str(item))
      extended_paths.append("*" + str(item))
  # for example "+2*2+4*4+7*7"

  # Generate different paths
  total_extended_paths = generate_ordered_subsets(extended_paths)
  total_extended_paths_strings = [''.join(t) for t in total_extended_paths]

  # paths that are possible
  total_viable_paths = []
  for token in total_extended_paths_strings:
    val = 0
    # 0, 2, 4...
    for i in range(0, len(token), 2):
      # add i
      if token[i] == '+':
        val += int(token[i+1])
      # multiply by i
      else:
        val *= int(token[i+1])
    if val <= 24:
      total_viable_paths.append(token)

  bestpath = ""
  # max score so far
  max_score = -1
  # for each path
  for path_item in total_viable_paths:
    # if the score is greater than the best score so far
    if compute_score(path_item) > max_score:
      # update score and path
      max_score = compute_score(path_item)
      bestpath = path_item

  return path, bestpath, max_score

In [15]:
print(predictor("2 4 7"))

('2 4 7', '+2+4*4', 83.8758)


In [19]:
print(predictor("1 4 5"))

('1 4 5', '+5*4+4', 84.9)


In [20]:
print(predictor("2 3 7"))

('2 3 7', '+3*2+2*3', 79.8236)


In [17]:
# Specify the file path
file_path = '/content/drive/MyDrive/Colab Notebooks/Active Site Data.xlsx'
# Read the Excel file
data_frame = pd.read_excel(file_path)

for index, row in data_frame.iterrows():
    # Access individual row values using row['column_name']
    path = row['First'], row['Second'], row['Third']
    print(predictor(str(path[0]) + " " + str(path[1]) + " " + str(path[2])))
# Display the data
#print(data_frame)

('1 2 3', '+1+3*3*2', 80.8086)
('1 2 4', '+2+4*4', 83.8758)
('1 2 6', '+2*2*6', 83.59299999999999)
('1 2 8', '+2*8+8', 84.05)
('1 2 9', '+9+1+2*2', 78.0806)
('1 3 4', '+4+1+3*3', 79.815)
('1 3 5', '+5+3*3', 82.41659999999999)
('1 3 6', '+3*6+6', 83.8586)
('1 3 7', '+7*3+3', 79.6112)
('1 3 8', '+3*8', 83.1642)
('1 4 5', '+5*4+4', 84.9)
('1 4 6', '+4*6', 84.65)
('1 4 7', '+4*4+1+7', 76.215)
('1 4 8', '+4*4+8', 81.8962)
('1 6 9', '+1*9+9+6', 79.543)
('2 3 4', '+2+4*4', 83.8758)
('2 3 5', '+5+3*3', 82.41659999999999)
('2 3 6', '+3*6+6', 83.8586)
('2 3 7', '+3*2+2*3', 79.8236)
('2 3 8', '+2*8+8', 84.05)
('2 3 9', '+9+3*2', 80.20859999999999)
('2 4 5', '+5*4+4', 84.9)
('2 4 6', '+4*6', 84.65)
('2 4 7', '+2+4*4', 83.8758)
('2 4 8', '+2*8+8', 84.05)
('2 4 9', '+2+4*4', 83.8758)
('2 5 6', '+2*2*6', 83.59299999999999)
('2 5 7', '+7+5*2', 80.72739999999999)
('2 5 8', '+2*8+8', 84.05)
('2 6 7', '+2*2*6', 83.59299999999999)
('2 6 8', '+2*8+8', 84.05)
('2 6 9', '+2*2*6', 83.59299999999999)
('2 7 8',